In [2]:
import cv2
import numpy as np
import math

In [3]:
cap = cv2.VideoCapture(0)

while(1):
    try:
        ret, frame = cap.read()
        frame=cv2.flip(frame,1)
        kernel = np.ones((3,3),np.uint8)
        
        roi=frame[100:300, 100:300]
        
        cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0)    
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    
        
        lower_skin = np.array([0,20,70], dtype=np.uint8)
        upper_skin = np.array([20,255,255], dtype=np.uint8)
    
        
        mask = cv2.inRange(hsv, lower_skin, upper_skin)
        mask = cv2.dilate(mask,kernel,iterations = 4)
        mask = cv2.GaussianBlur(mask,(5,5),100) 
    
        
        contours,hierarchy= cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
        
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        
        
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)
        #mask = cv2.dilate(mask, kernel, iterations=1)
        #res=cv2.bitwise_and(img,img,mask=mask)
        hull = cv2.convexHull(cnt)
        
        
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)
        arearatio=((areahull-areacnt)/areacnt)*100
    
        hull = cv2.convexHull(approx, returnPoints=False)
        defects = cv2.convexityDefects(approx, hull)
        l=0
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0])
            pt= (100,180)
        
            
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            
            
            d=(2*ar)/a
            
            
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            
        
            
            if angle <= 90 and d>30:
                l += 1
                cv2.circle(roi, far, 3, [255,0,0], -1)
            
            cv2.line(roi,start, end, [0,255,0], 2)   
        
        l+=1
    
        font = cv2.FONT_HERSHEY_SIMPLEX
    
        if l==1:
            if areacnt<2000:
                cv2.putText(frame,'Welcome to Projector Control using gestures')
            else:
                if arearatio<12:
                    cv2.putText(frame,'Previous',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        
        else:
            cv2.putText(frame,'Next',(10,50),font,2,(0,0,255),3,cv2.LINE_AA)
    
        cv2.imshow('mask',mask)
        cv2.imshow('frame',frame)
        #cv2.imshow('res',res)
    except:
        pass
    
    k=cv2.waitKey(1) & 0xFF
    if k == 32:
        break
        
cv2.destroyAllWindows()
cap.release()         

In [ ]:
import numpy as np
import cv2
import sys
from pynput.keyboard import Key,Controller
import time
#from keras.preprocessing import image
#from PIL import Image

In [ ]:
time.sleep(10)
hand_cascade_palm=cv2.CascadeClassifier('palm.xml')
hand_cascade_fist=cv2.CascadeClassifier('fist.xml')
keyboard=Controller()

cap=cv2.VideoCapture(1)
while(1):
    ret,frame=cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    hands=hand_cascade_palm.detectMultiScale(gray,scaleFactor=1.3,
                                       minNeighbors=5)
    
    fist=hand_cascade_fist.detectMultiScale(gray,scaleFactor=1.3,
                                           minNeighbors=5)
    
    for (x,y,w,h) in hands:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)
        cv2.putText(frame,'NEXT SLIDE',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
        while(1):
            keyboard.press(Key.right)
            keyboard.release(Key.right)
            time.sleep(1.5)
            break
        
    
    for(x,y,w,h) in fist:
        cv2.rectangle(frame,(x,y),(w+x,y+h),(0,255,0),3)
        cv2.putText(frame,'PREVIOUS SLIDE',(0,50), font, 2, (0,255,0), 3, cv2.LINE_AA)
        while(1):
            keyboard.press(Key.left)
            keyboard.release(Key.left)
            time.sleep(1.5)
            break
                
    
    cv2.imshow('hands',frame)
    #cv2.imshow('gray',gray)
    
    k=cv2.waitKey(1)& 0xff
    if k==27:
        break
cv2.destroyAllWindows()
cap.release()